# MLP with raw audio data
Used to generate and save a MLP model that solely utilizes raw audio data.

Requires data from:
* `generate_data_py.ipynb`

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import librosa
import torch
import torch.nn as nn

import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

import pyloudnorm as pyln

# Embedding
import umap

In [37]:
# load up raw data
raw_labels = np.load(os.path.join("data", "labels.npy"))
raw_data = np.load(os.path.join("data", "raw_data.npy"))
print(raw_labels)
print(raw_labels.shape)
print(raw_data.shape)

[1 1 1 ... 0 0 0]
(1046,)
(1046, 220500)


Embed the raw data using best UMAP model

In [50]:
embedding_dim = 64

optimal_param = {'metric': 'correlation',
                 'min_dist': 0.0,
                 'n_components': embedding_dim,
                 'n_neighbors': 100}

In [51]:
umap_model = umap.UMAP(**optimal_param)
umap_model

UMAP(metric='cosine', min_dist=0.0, n_components=64, n_neighbors=50)

In [52]:
embedding_data = umap_model.fit_transform(raw_data) # Unsupervised 

In [53]:
batchSize = 30
embedding_train, embedding_test, labels_train, labels_test = train_test_split(embedding_data, raw_labels, test_size=0.2, random_state=100)

In [55]:
# Embedded raw data 
print(f"Dimensionality reduction (embedding dim={embedding_dim}):")
print(f"  Train: ({embedding_train.shape[0]}, {raw_data.shape[1]}) -> {embedding_train.shape}")
print(f"  Test: ({embedding_test.shape[0]}, {raw_data.shape[1]}) -> {embedding_test.shape}")

Dimensionality reduction (embedding dim=64):
  Train: (836, 220500) -> (836, 64)
  Test: (210, 220500) -> (210, 64)


In [56]:
embedding_train_tensor    = torch.tensor(embedding_train).unsqueeze(1)
embedding_test_tensor     = torch.tensor(embedding_test).unsqueeze(1)
labels_train_tensor = torch.tensor(labels_train)
labels_test_tensor  = torch.tensor(labels_test)

train_dataset = TensorDataset(embedding_train_tensor, labels_train_tensor)
test_dataset  = TensorDataset(embedding_test_tensor, labels_test_tensor)

train_loader  = DataLoader(train_dataset, batch_size = batchSize, shuffle = True)
test_loader   = DataLoader(test_dataset, batch_size = batchSize, shuffle = False)

In [57]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)  
        self.fc5 = nn.Linear(128, 64)   
        self.output = nn.Linear(64, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.output(x)
        return x

In [58]:
# The Model
model = MLP(input_dim=embedding_dim, output_dim=1)

# Loss Function and Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [59]:
total_num_params = 0
for parameter in model.parameters():
    # print(parameter.shape)
    for p in parameter.shape:
        total_num_params += p
print(f"Total number of parameters in MLP: {total_num_params}")

Total number of parameters in MLP: 6018


In [60]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.float()  
        labels = labels.float() 
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        
    epoch_loss = running_loss / len(train_loader.dataset)
    
    if epoch % (num_epochs // 10) == 0 or epoch == num_epochs - 1:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

Epoch [1/100], Loss: 0.6935
Epoch [11/100], Loss: 0.3523
Epoch [21/100], Loss: 0.3479
Epoch [31/100], Loss: 0.3709
Epoch [41/100], Loss: 0.3357
Epoch [51/100], Loss: 0.3479
Epoch [61/100], Loss: 0.3402
Epoch [71/100], Loss: 0.3330
Epoch [81/100], Loss: 0.3801
Epoch [91/100], Loss: 0.3630
Epoch [100/100], Loss: 0.3444


In [61]:
def evaluate_model(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    true_labels = []
    predictions = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.float()
            labels = labels.float()
            outputs = model(inputs)
            predicted_probs = torch.sigmoid(outputs)  # Sigmoid activation to get probabilities
            predicted_labels = (predicted_probs > 0.5).float()  # Threshold probabilities to get binary labels
            true_labels.extend(labels.tolist())
            predictions.extend(predicted_labels.squeeze().tolist())

    # Calculate metrics
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    auroc = roc_auc_score(true_labels, predictions)

    return accuracy, f1, precision, recall, auroc


accuracy, f1, precision, recall, auroc = evaluate_model(model, test_loader)
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"AUROC: {auroc:.4f}")

Accuracy: 0.8810
F1 Score: 0.8927
Precision: 0.9123
Recall: 0.8739
AUROC: 0.8820


In [ ]:
# Save Model
torch.save(model.state_dict(), os.path.join("data", "mlp_embedding_model_batch_100_epoch_5.pth"))

# Grid Search

In [65]:
nn_range = [5, 50, 100, 150, 250, 500]

In [66]:
embedding_dim_range = [8, 32, 64, 128, 256]

In [95]:
distance_metrics = [
    "cosine",
    "correlation",
]

In [96]:
from tqdm import tqdm
from sklearn.model_selection import ParameterGrid

In [ ]:
umap_embeddings = []
all_parameters = []
for metric_range in distance_metrics:
    param_grid_umap = {
        'n_components': embedding_dim_range,
        'min_dist': [0.0],
        'n_neighbors': nn_range,
        'metric': [metric_range] 
    }
    grid_umap = list(ParameterGrid(param_grid_umap))

    params_combined = grid_umap

    all_parameters += params_combined

    # project datapoints to embedding
    for params in tqdm(grid_umap):
        model = umap.UMAP(**params)
        embedding = model.fit_transform(raw_data)
        umap_embeddings.append(embedding)


 27%|███████████▍                               | 8/30 [21:49<59:47, 163.09s/it]

In [81]:
statistics = []
for param_idx, embedding_data in tqdm(enumerate(umap_embeddings)):

    print(params_combined[param_idx])
    
    embedding_dim = params_combined[param_idx]['n_components']
    
    
    batchSize = 30
    embedding_train, embedding_test, labels_train, labels_test = train_test_split(embedding_data, raw_labels, test_size=0.2, random_state=100)
    
    # Embedded raw data 
    print(f"Dimensionality reduction (embedding dim={embedding_dim}):")
    print(f"  Train: ({embedding_train.shape[0]}, {raw_data.shape[1]}) -> {embedding_train.shape}")
    print(f"  Test: ({embedding_test.shape[0]}, {raw_data.shape[1]}) -> {embedding_test.shape}")
    
    embedding_train_tensor    = torch.tensor(embedding_train).unsqueeze(1)
    embedding_test_tensor     = torch.tensor(embedding_test).unsqueeze(1)
    labels_train_tensor = torch.tensor(labels_train)
    labels_test_tensor  = torch.tensor(labels_test)
    
    train_dataset = TensorDataset(embedding_train_tensor, labels_train_tensor)
    test_dataset  = TensorDataset(embedding_test_tensor, labels_test_tensor)
    
    train_loader  = DataLoader(train_dataset, batch_size = batchSize, shuffle = True)
    test_loader   = DataLoader(test_dataset, batch_size = batchSize, shuffle = False)

    # The Model
    model = MLP(input_dim=embedding_dim, output_dim=1)
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)


    num_epochs = 100
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs = inputs.float()  
            labels = labels.float() 
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            
        epoch_loss = running_loss / len(train_loader.dataset)
        
        if epoch % (num_epochs // 10) == 0 or epoch == num_epochs - 1:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    
    accuracy, f1, precision, recall, auroc = evaluate_model(model, test_loader)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"AUROC: {auroc:.4f}")

    statistics.append([accuracy, f1, precision, recall, auroc])

0it [00:00, ?it/s]

{'metric': 'correlation', 'min_dist': 0.0, 'n_components': 8, 'n_neighbors': 5}
Dimensionality reduction (embedding dim=8):
  Train: (836, 220500) -> (836, 8)
  Test: (210, 220500) -> (210, 8)
Epoch [1/100], Loss: 0.6653
Epoch [11/100], Loss: 0.4601
Epoch [21/100], Loss: 0.4430
Epoch [31/100], Loss: 0.4272
Epoch [41/100], Loss: 0.4193
Epoch [51/100], Loss: 0.4144
Epoch [61/100], Loss: 0.4070
Epoch [71/100], Loss: 0.3886
Epoch [81/100], Loss: 0.3869
Epoch [91/100], Loss: 0.3868


1it [00:15, 15.79s/it]

Epoch [100/100], Loss: 0.3861
Accuracy: 0.8143
F1 Score: 0.8421
Precision: 0.8125
Recall: 0.8739
AUROC: 0.8051
{'metric': 'correlation', 'min_dist': 0.0, 'n_components': 8, 'n_neighbors': 50}
Dimensionality reduction (embedding dim=8):
  Train: (836, 220500) -> (836, 8)
  Test: (210, 220500) -> (210, 8)
Epoch [1/100], Loss: 0.7044
Epoch [11/100], Loss: 0.4279
Epoch [21/100], Loss: 0.5125
Epoch [31/100], Loss: 0.3945
Epoch [41/100], Loss: 0.3930
Epoch [51/100], Loss: 0.3968
Epoch [61/100], Loss: 0.3684
Epoch [71/100], Loss: 0.3602
Epoch [81/100], Loss: 0.3687
Epoch [91/100], Loss: 0.3579


2it [00:31, 15.68s/it]

...

Epoch [100/100], Loss: 0.6915
Accuracy: 0.5667
F1 Score: 0.7234
Precision: 0.5667
Recall: 1.0000
AUROC: 0.5000
{'metric': 'correlation', 'min_dist': 0.0, 'n_components': 256, 'n_neighbors': 250}
Dimensionality reduction (embedding dim=256):
  Train: (836, 220500) -> (836, 256)
  Test: (210, 220500) -> (210, 256)
Epoch [1/100], Loss: 0.6975
Epoch [11/100], Loss: 0.6916
Epoch [21/100], Loss: 0.6916
Epoch [31/100], Loss: 0.6916
Epoch [41/100], Loss: 0.6917
Epoch [51/100], Loss: 0.6916
Epoch [61/100], Loss: 0.6915
Epoch [71/100], Loss: 0.6915
Epoch [81/100], Loss: 0.6916
Epoch [91/100], Loss: 0.6915


29it [08:15, 18.55s/it]

Epoch [100/100], Loss: 0.6916
Accuracy: 0.5667
F1 Score: 0.7234
Precision: 0.5667
Recall: 1.0000
AUROC: 0.5000
{'metric': 'correlation', 'min_dist': 0.0, 'n_components': 256, 'n_neighbors': 500}
Dimensionality reduction (embedding dim=256):
  Train: (836, 220500) -> (836, 256)
  Test: (210, 220500) -> (210, 256)
Epoch [1/100], Loss: 0.6981
Epoch [11/100], Loss: 0.6918
Epoch [21/100], Loss: 0.6916
Epoch [31/100], Loss: 0.6916
Epoch [41/100], Loss: 0.6917
Epoch [51/100], Loss: 0.6916
Epoch [61/100], Loss: 0.6916
Epoch [71/100], Loss: 0.6916
Epoch [81/100], Loss: 0.6916
Epoch [91/100], Loss: 0.6919


30it [08:34, 17.14s/it]

Epoch [100/100], Loss: 0.6915
Accuracy: 0.5667
F1 Score: 0.7234
Precision: 0.5667
Recall: 1.0000
AUROC: 0.5000


In [86]:
correlation_statistics = np.array(statistics)

In [91]:
correlation_statistics

array([[0.81428571, 0.84210526, 0.8125    , 0.87394958, 0.80510666],
       [0.80952381, 0.80769231, 0.94382022, 0.70588235, 0.82546865],
       [0.8       , 0.8173913 , 0.84684685, 0.78991597, 0.80155139],
       [0.88095238, 0.90118577, 0.85074627, 0.95798319, 0.86910149],
       [0.88095238, 0.90118577, 0.85074627, 0.95798319, 0.86910149],
       [0.83333333, 0.86166008, 0.81343284, 0.91596639, 0.82062056],
       [0.7952381 , 0.81545064, 0.83333333, 0.79831933, 0.79476406],
       [0.88571429, 0.89655172, 0.92035398, 0.87394958, 0.88752424],
       [0.86666667, 0.87719298, 0.91743119, 0.84033613, 0.87071752],
       [0.86666667, 0.89230769, 0.82269504, 0.97478992, 0.85003232],
       [0.84761905, 0.85840708, 0.90654206, 0.81512605, 0.85261797],
       [0.86190476, 0.88803089, 0.82142857, 0.96638655, 0.84583064],
       [0.82380952, 0.85375494, 0.80597015, 0.90756303, 0.81092437],
       [0.88095238, 0.89361702, 0.90517241, 0.88235294, 0.88073691],
       [0.8952381 , 0.91129032, 0.